In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

In [ ]:
%pip install python-chess
%pip install import_ipynb

^C
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/venv/lib/python3.7/site-packages/pip/__main__.py", line 23, in <module>
    from pip._internal.cli.main import main as _main  # isort:skip # noqa
  File "/root/venv/lib/python3.7/site-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/root/venv/lib/python3.7/site-packages/pip/_internal/cli/autocompletion.py", line 9, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/root/venv/lib/python3.7/site-packages/pip/_internal/cli/main_parser.py", line 7, in <module>
    from pip._internal.cli import cmdoptions
  File "/root/venv/lib/python3.7/site-packages/pip/_internal/cli/cmdoptions.py", line 24, in <module>
    from pip._internal.cli.p

In [ ]:
import chess
import json

import import_ipynb
import Game
import Globals
from MinimaxAlgorithm import make_move_minimax, make_move_minimax_memoization
from RandomAlgorithm import make_random_move
from AlphaBetaAlgorithm import make_move_alphabeta, make_move_alphabeta_iterative_deepening




importing Jupyter notebook from Game.ipynb


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
importing Jupyter notebook from Globals.ipynb


importing Jupyter notebook from MinimaxAlgorithm.ipynb


importing Jupyter notebook from Util.ipynb


importing Jupyter notebook from RandomAlgorithm.ipynb


importing Jupyter notebook from AlphaBetaAlgorithm.ipynb


In genreal, a game of chess can be initialized through the instance of a Game-class. To be precise, the creation of the game-object can be "customized" with the following parameters:

    game = Game.Game(  
                make_move_algorithm_white = " desired_algo or None (human player) " ,  
                make_move_algorithm_black = " desired_algo or None (human player) " ,  
                opening_book              = " Path to opening_book-library or None"        ,  
                endgame_tablebase_dir     = " Path to endgame_tablebase or None"  
                )

in this case, the make_move_algorithm white and black represent the different algorithms which can play against each other or against a human enemy. In case a human wants to play against an algorithm, simply set 

    make_move_algorithm = None
    
as descriped above (Leaving out the argument has the same effect). 
The possible algorithms are:  

1. make_random_move
2. make_move_minimax
3. make_move_minimax_memoization
4. make_move_alphabeta
5. make_move_alphabeta_iterative_deepening  

Furthermore, it is recommended to use both the opening_book as well as the endgame_tablebase_dir argument as not None in order to boost efficiency and reduce the time needed by the AI to make a move.

Here is a basic game which can be played. The ai's are represented through the minimax-algorithm and the alphabeta-algorithm. In this case, the endgame-table is not implemented. 

In [ ]:
game = Game.Game(make_move_minimax, make_move_alphabeta, endgame_tablebase_dir= None)
#game.play()

___
This is for debbuging purposes. In the following cell we can see multiple tests executed in a loop. Here, all possible tests are run.


In [ ]:
chess_problems = Globals.chess_problems
for c in chess_problems:
    game.debug_play(c, automation = True)

AttributeError: module 'Globals' has no attribute 'chess_problems'

In [ ]:
game.debug_play('BWS', automation=False)


AJW, 
AF, 
LE, 
MPN, 
EK, 
GBE


KeyboardInterrupt: Interrupted by user

In the following cell the test-details of all previews tests are displayed.

In [ ]:
import pprint #importing prettyprint for prettier print :D

pprint.pprint(game.auto_tests)
print(f'\n{game.wins}')

{'AF': {1: (10000000, 'c7b8'), 2: (-10000000, 'g6h8'), 3: (10000000, 'f7f4')},
 'AJW': {1: (10000000, 'b1f1'),
         2: (-10000000, 'h8f7'),
         3: (10000000, 'h5f7'),
         4: (-10000000, 'd5c6'),
         5: (10000000, 'f1g2')},
 'EK': {1: (135, 'f1e3'),
        2: (-25, 'g1h1'),
        3: (115, 'g2f4'),
        4: (-80, 'h1g1'),
        5: (100, 'c4d5'),
        6: (-90, 'g1h1'),
        7: (90, 'e3f5'),
        8: (10, 'h1g1'),
        9: (90, 'c3c6'),
        10: (-80, 'g1f1'),
        11: (110, 'c6e6'),
        12: (-100, 'f1g1'),
        13: (100, 'e6f6'),
        14: (0, 'g1h1'),
        15: (10000000, 'c2c1'),
        16: (-10000000, 'h1h2'),
        17: (10000000, 'f6h6')},
 'GBE': {1: (75, 'a2b2'),
         2: (-50, 'b3c4'),
         3: (110, 'g8f7'),
         4: (880, 'g1c5'),
         5: (-380, 'b2b1'),
         6: (860, 'h1b1'),
         7: (-250, 'a1b1'),
         8: (1580, 'h2h1q'),
         9: (-740, 'b1c2'),
         10: (990, 'c5f8'),
         11: (105, '

In [ ]:
game.debug

{1: (75, 'a2b2'),
 2: (-50, 'b3c4'),
 3: (110, 'g8f7'),
 4: (880, 'g1c5'),
 5: (-380, 'b2b1'),
 6: (860, 'h1b1'),
 7: (-250, 'a1b1'),
 8: (1580, 'h2h1q'),
 9: (-740, 'b1c2'),
 10: (990, 'c5f8'),
 11: (105, 'b4d3'),
 12: (25, 'c4d4'),
 13: (145, 'a6a7'),
 14: (110, 'h1h7'),
 15: (80, 'c2d2'),
 16: (120, 'f8d6'),
 17: (125, 'a7d7'),
 18: (80, 'd6f8'),
 19: (125, 'a4c3'),
 20: (40, 'f8b4'),
 21: (330, 'd3b4'),
 22: (270, 'd4c4'),
 23: (100, 'b4d3'),
 24: (25, 'h7f5'),
 25: (600, 'd7b7'),
 26: (-590, 'f5d3'),
 27: (920, 'e2d3'),
 28: (0, 'c4d4'),
 29: (105, 'c3e2'),
 30: (-30, 'd4e5'),
 31: (140, 'd2c3'),
 32: (-40, 'e5f5'),
 33: (140, 'c3d4'),
 34: (-60, 'f5g5'),
 35: (130, 'd4e5'),
 36: (-110, 'g5h6'),
 37: (110, 'e5f4'),
 38: (-10, 'h6g7'),
 39: (30, 'f4e5'),
 40: (-10, 'g7h6'),
 41: (30, 'f3f4'),
 42: (-20, 'h6h7'),
 43: (30, 'f4f5'),
 44: (0, 'h7h6'),
 45: (10, 'f7e6'),
 46: (0, 'h6h5'),
 47: (20, 'b7g7'),
 48: (-10, 'h5h6'),
 49: (40, 'f5f6'),
 50: (-30, 'h6h5'),
 51: (770, 'f6f7'),


___

In the following case, a chess_problem is given in the function call. After that there is an example of a false argument

In [ ]:
game.debug_play(chess_problem = "EK")

No endgame tablebase available


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-e00d88747fca>", line 1, in <module>
    game.debug_play(chess_problem = "EK")
  File "<string>", line 16, in debug_play
  File "<string>", line 13, in display_debug
  File "/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 851, in raw_input
    password=False,
  File "/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 892, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/shared-libs/python3

TypeError: object of type 'NoneType' has no len()

In [ ]:
game.debug_play(chess_problem = "H")


KeyboardInterrupt: Interrupted by user

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>